In [1]:
# -*- coding: utf-8 -*-
"""
2018-06-28,29

Blinker analysis: PL intensity versus laser power
PL intensity versus laser power, QE 4-22-2018.11,00am

@author: Viva Horowitz
"""

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import datetime
import seaborn as sns
import numpy as np
import sys
sys.path.append(r'C:\Users\Viva\Documents')
sys.path.append(r'C:\Users\Viva\Documents\detect_peaks.py')
import detect_peaks
## http://nbviewer.jupyter.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
from detect_peaks import detect_peaks


sns.set(style='ticks')
sns.set_context('poster')

%pylab inline
from matplotlib.mlab import frange

# ** set for each dataset
datadate = '2018-06-01'
datafolder = r'O:\User\Viva\data\2018-06-01_blinking-statistics\4-22-2018.11,00am'
analysisfolder = r'C:\Users\Viva\Documents\analysis\blinker_analysis\power_dependence,QE4-22-2018.11,00am'
filenames = ['4-22-2018.11,00am, 03, ND2,4.txt',
             '4-22-2018.11,00am, 04, ND2,3.txt',
             '4-22-2018.11,00am, 05, ND2,2.txt',
             '4-22-2018.11,00am, 06, ND2,0.txt',
             '4-22-2018.11,00am, 07, ND1,6.txt',
             '4-22-2018.11,00am, 08, ND1,5.txt',
             '4-22-2018.11,00am, 11, ND1,4.txt',
             '4-22-2018.11,00am, 12, ND1,3.txt',
             '4-22-2018.11,00am, 13, ND1,2.txt',
             '4-22-2018.11,00am, 14, ND1,0.txt']   
dataname = 'QE 4-22-2018.11,00am'
NDlist = [2.4, 2.3, 2.2, 2.0, 1.6, 1.5, 1.4, 1.3, 1.2, 1.0]

print(np.__version__)
print(sns.__version__)
print(pd.__version__)
print(mpl.__version__)

Populating the interactive namespace from numpy and matplotlib
1.10.4
0.8.0
0.20.3
2.1.0


In [2]:
os.chdir(r'O:\User\Viva')
filterwheel = pd.read_csv('Josh_filter_wheel.csv')

In [3]:
# Does not display all digits but they are stored.
filterwheel.head(1)

,ND,Power (W)
0,1.0,0.000995


In [4]:
## https://stackoverflow.com/questions/32765333/how-do-i-replicate-this-matlab-function-in-numpy/32765547#32765547
def histc(X, bins):
    map_to_bins = np.digitize(X,bins)
    r = np.zeros(bins.shape)
    for i in map_to_bins:
        r[i-1] += 1
    return [r, map_to_bins]

def manualnorm(histdata, binwidth):
    return (1/(histdata.sum()*binwidth))*histdata

def arangebylength(start, step, length):
    return np.arange(start = start, stop=start+step*(length-.5), step = step )

# rebin: combine two rows into one.
def rebindata(data, colname, timecolname):
    first = 1 # this is a value that indicates if we're on the first (=1) or the second (-1) of a pair of two bins
    rebinvals = []

    print data.tail()
    for value in data[colname]:
        #print 'value: ' + str(value)
        if first == 1:
            prevval = value
        else:
            # calculate average of the two bins
            combined = (prevval + value) /2.0 
            rebinvals.append(combined)
            del prevval
        first = -1 * first
         
    timesetting = data[timecolname][1]-data[timecolname][0]
    rebintimes = arangebylength(start =0, step = timesetting*2, length = len(rebinvals) )
    datadict = {timecolname: rebintimes, colname: rebinvals}
    rebinneddata = pd.DataFrame( data = datadict)
    return rebinneddata

In [5]:
os.chdir(datafolder)

tname = 'Time (s)'
PLname = 'PL Intensity (kCts/s)'

rebins_to_do = 0 # number of times to rebin. **
newfilenames = copy(filenames)
timesettings = copy(NDlist)
fps= zeros_like(NDlist)

for i in range(len(filenames)):
    data = pd.read_csv(filenames[i],
                       skiprows=1,
                       header=None,
                       sep='\t',
                       names=[tname + ' ' + str(NDlist[i]),str(NDlist[i])],
                       usecols=[0,1])
    #data = data.set_index(tname)
    print(data.head())
    rebincount = 0
    
    # *** run this an appropriate number of times, can skip if not rebinning
    origfilenames = filenames
    timecolname = tname + ' ' + str(NDlist[i])
    for j in range(rebins_to_do):
        data = rebindata(data, colname=str(NDlist[i]), timecolname=timecolname)
        rebincount = rebincount + 1
    timesettings[i] = data[timecolname][1]-data[timecolname][0]
    fps[i] = 1/timesettings[i]
    print('Effective collection rate: ' + str(fps[i]) + ' Hz')
    print('Time bin size: ' + str(timesettings[i]) + ' sec')
    newfilenames[i] = origfilenames[i] + ', ' + str(fps[i]) + ' Hz'

    #plt.scatter(x=data['Time (s) ' + str(NDlist[i])], y=str(NDlist[i]),linewidth=0.3)
    data[str(NDlist[i])].plot(linewidth=0.3)
    plt.ylabel('photoluminescence (kCts/s)')
    #plt.xlabel('Time(s)')
    plt.title(newfilenames[i])
    plt.show()
    
    if i > 0:
        alldata = pd.concat([alldata,data], axis=1)
    else:
        alldata = data        
 
dataname = dataname + ',' + str(rebins_to_do) + ' rebins'
#filename = filename.replace('\\','-')
#origfilename = filename
os.chdir(analysisfolder)
today = datetime.date.isoformat(datetime.date.today())
if not os.path.exists('taken-' + datadate):
    os.makedirs('taken-' + datadate)
os.chdir('taken-' + datadate)

print('today: '+today)
os.getcwd()

   Time (s) 2.4   2.4
0         0.008  5.75
1         0.012  6.25
2         0.016  5.50
3         0.020  5.75
4         0.024  6.25


NameError: name 'timecolname' is not defined

In [ ]:
alldata.head()

In [ ]:
alldata.tail()

In [ ]:
strNDlist = []
for ND in NDlist:
    strNDlist.append(str(ND))
data = alldata[strNDlist]
data.head()

In [ ]:
strNDlist

In [ ]:
print max(data.max())
print min(data.min())

In [ ]:
stepsize = 1
# adding a stepsize to the max is a workaround to a bug
binsequence= frange(min(data.min()),max(data.max())+stepsize,stepsize)

try:
    del histdata
except NameError:
    pass
#histdata=pd.DataFrame(columns=['ND', PLname, "Occurrences"])

histdict = []
for i in range(len(strNDlist)):
    plt.hist(data[strNDlist[i]], bins=binsequence,  histtype='step', linewidth=1, label=strNDlist[i], normed=True)
    newhistdata, toss = histc(data[strNDlist[i]], bins=np.asarray(binsequence))
    NDlisttemp = [NDlist[i]] * len(binsequence)
    ## Norm the newhistdata
    #   adding a stepsize to the max was a workaround to a bug
    #   now we remove that last bin, which should have been empty (but it's so buggy it wasn't empty)
    newhistdatanormed = manualnorm(newhistdata[:-1], stepsize)
    
    try:
        # append histogram to dataframe histdata

        histdata = histdata.append(pd.DataFrame(transpose([NDlisttemp[:-1], binsequence[:-1],newhistdatanormed.tolist()])))
#        print "appended"
    except NameError:
        histdata = pd.DataFrame(transpose([NDlisttemp[:-1], binsequence[:-1],newhistdatanormed.tolist()]))
        print "histdata initialized"
histdata = histdata.rename(index=str, columns={0: 'ND', 1: PLname, 2:"Probability"})           
tabledata = histdata.pivot("ND", PLname, "Probability")

# In terms of laser intensity instead of optical density
powertabledata = tabledata.join(filterwheel.set_index('ND')).set_index('Power (W)')

#sns.distplot(data[strNDlist[9]], bins=numbins, kde=True, rug=False, hist_kws={"histtype": "step", "linewidth": 1, "alpha": 1, "color": "k"});
plt.xlabel('PL (kCts/s)')
plt.ylabel('Occurrences (normed)')
#ax = plt.gca()
#ax.set_yscale('symlog', linthreshy=1)
#ax.set_xscale('log')
sns.despine()
plt.legend()
plt.show()


In [ ]:
NDlist

In [ ]:
## I want to extract the peaks of these data
# https://stackoverflow.com/questions/37374983/get-data-points-from-seaborn-distplot

i = 5

# bw = bandwidth for calculating kde
bw = 2
# mph = minimum peak height
mph = .001
# mpd = minimum peak distance
mpd = 5

plt.figure()
output1 = sns.distplot(data[strNDlist[i]],bins=binsequence,kde_kws={'bw':bw, 'label':'KDE'},).get_lines()[0].get_data()

## http://nbviewer.jupyter.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb



ind= detect_peaks(output1[1], valley=False, show=True, mph=mph, mpd=mpd)
#print ind

figure()
plot(output1[0], output1[1])
plot(output1[0][ind], output1[1][ind], 'r.')
output1[0][ind]

In [ ]:
os.chdir(analysisfolder)
today = datetime.date.isoformat(datetime.date.today())
if not os.path.exists('taken-' + datadate):
    os.makedirs('taken-' + datadate)
os.chdir('taken-' + datadate)

print('today: '+today)
print os.getcwd()

peaksPL = {NDlist[0]:None}
peaksprob = {NDlist[0]:None}

plt.figure(figsize=(5,10))
title(dataname)
for i in range(len(strNDlist)):
    print str(i) + ': ' + strNDlist[i]
    ax = plt.subplot(len(strNDlist), 1, i+1)
    output = sns.distplot(data[strNDlist[i]],
                          bins=binsequence, 
                          label='ND ' + strNDlist[i],
                          kde_kws={'bw':bw}).get_lines()[0].get_data()
    ind= detect_peaks(output[1], valley=False, show=False, mph=mph, mpd=mpd)
    ax.plot(output[0][ind], output[1][ind], 'r.')
    peaksPL[NDlist[i]] = output[0][ind]
    peaksprob[NDlist[i]] =output[1][ind]
    legend()

xlabel(PLname);

plt.savefig(today + ',' + dataname+',PLvspowerhists.pdf', bbox_inches='tight')
plt.savefig(today + ',' + dataname+',PLvspowerhists.png', bbox_inches='tight', dpi=200)

In [ ]:
peaksPL

In [ ]:
peaksprob

In [ ]:
# The bug!
plot(newhistdata);

In [ ]:
# Dealt with the bug!
plot(newhistdatanormed);

In [ ]:
histdata.head()

In [ ]:
NDlist

In [ ]:
powlist = []
for ND in NDlist:
    powlist.append(filterwheel.loc[filterwheel['ND']==ND]['Power (W)'].values[0]) 

powlist

In [ ]:
#filterwheel.set_index('ND')

# previously made NDlistwrap manually
"""
NDlistwrap = [2.45, 2.35, 2.25, (2.2+2.0)/2, (2.0+1.6)/2, 1.55, 1.45, 1.35, 1.25, (1.2+1.0)/2, 1.0-0.05]

NDlistwrap[::-1]
"""

In [ ]:
# create a listwrap that wraps around the list 
# This is what I need for pcolormesh.
def listwrap(currentlist):
    listwrap = [0] * (len(currentlist)+1)
    for i in range(len(currentlist)):
        try:
            listwrap[i+1] = ((currentlist[i]+currentlist[i+1])/2)
        except IndexError:
            pass
    topstep = listwrap[2]-listwrap[1]
    listwrap[0]=listwrap[1]-topstep
    botstep = listwrap[-2]-listwrap[-3]
    listwrap[-1]=listwrap[-2]+botstep
    return listwrap

In [ ]:
NDlistwrap = listwrap(NDlist)
NDlistwrap[::-1]

In [ ]:
tabledata

In [ ]:
peaksPL

In [ ]:
peaksPL.get(2.4)

In [ ]:
cmap = sns.cubehelix_palette(light=1, as_cmap=True)
#cmap = sns.color_palette("Grey", as_cmap=True)

plt.figure(figsize=(6,4))
plt.pcolormesh(NDlistwrap[::-1], binsequence, transpose(tabledata), cmap=cmap)
sns.despine()
colorbar()

for i in range(len(NDlist)):
    plot([NDlist[i]]*len(peaksPL.get(NDlist[i])),peaksPL.get(NDlist[i]), '.', color='grey')

ylim(ymin=0)
plt.ylabel(PLname)
plt.xlabel('ND')

In [ ]:
powlist

In [ ]:
powlistwrap = listwrap(powlist)
powlistwrap

In [ ]:
lowerpeaks = np.zeros_like(NDlist)
keepgoing = [False] * len(NDlist)
for i in range(len(NDlist)):
#    print peaksPL.get(NDlist[i]).min()
    lowerpeaks[i] = peaksPL.get(NDlist[i]).min()
    # set keepgoign to true if there  is still another value
lowerpeaks

In [ ]:
#collect the upperpeaks

upperpeaks = np.zeros_like(NDlist)
for i in range(len(NDlist)):
#    print peaksPL.get(NDlist[i]).min()
    upperpeaks[i] = peaksPL.get(NDlist[i]).max()
    # set keepgoign to true if there  is still another value
# *** not quite automated
upperpeaks = upperpeaks[:-2]
powlistforupperpeaks = powlist[:-2]
upperpeaks

In [ ]:
from scipy.optimize import curve_fit
import scipy
print(scipy.__version__)
# need at least version 14.0 of scipy.

def expfit(t, A, tau):
    return (A * np.exp(t/(-tau)))

def powlaw(t, C, m):
    return C * t**(-m)

# laser power P, saturation power Psat
def satcurve(P,Psat,signalmax):
    return signalmax/(1 + Psat/P)

In [ ]:
powlist

In [ ]:
x = np.linspace(start = min(powlistwrap), stop = max(powlistwrap),num =100)
x_extended = np.linspace(start = min(powlistwrap), stop = 30*max(powlistwrap),num =300)

# curvefit
lowerpeaksparam, lowerpeakspcov = curve_fit(satcurve, xdata=powlist, ydata=lowerpeaks,p0 = None)
lowerpeaksfit = satcurve(x,lowerpeaksparam[0],lowerpeaksparam[1])
lowerpeaksfitextended = satcurve(x_extended,lowerpeaksparam[0],lowerpeaksparam[1])

print 'Lower peaks saturation at ' + str(lowerpeaksparam[0]) + ' +/- ' + str(np.sqrt(lowerpeakspcov[0,0])) + ' W'

# curvefit upperpeaks
upperpeaksparam, upperpeakspcov = curve_fit(satcurve, xdata=powlistforupperpeaks, ydata=upperpeaks,p0 = None)
upperpeaksfit = satcurve(x,upperpeaksparam[0],upperpeaksparam[1])
upperpeaksfitextended = satcurve(x_extended,upperpeaksparam[0],upperpeaksparam[1])

print 'Upper peaks saturation at ' + str(upperpeaksparam[0]) + ' +/- ' + str(np.sqrt(upperpeakspcov[0,0])) + ' W'


figure(figsize=(6,4))
for i in range(len(NDlist)):
    plot([powlist[i]]*len(peaksPL.get(NDlist[i])),peaksPL.get(NDlist[i]), '.', color='r') # plot peaks
plot(x_extended,lowerpeaksfitextended, '--', color='grey') # plot curvefit out a bit further
plot(x,lowerpeaksfit, 'k') # plot curvefit where I actually did the curvefit.
plot(x_extended,upperpeaksfitextended, '--', color='grey') # plot curvefit out a bit further
plot(x,upperpeaksfit, 'k') # plot curvefit where I actually did the curvefit.
xlabel('Laser power (W)')
ylabel(PLname)
plt.axvline(upperpeaksparam[0], label="upper peaks saturation", color='g', alpha=.2)
plt.axvline(lowerpeaksparam[0], label="lower peaks saturation", alpha=.2)
plt.axhline(upperpeaksparam[1], color='g', alpha=.2)
plt.axhline(lowerpeaksparam[1], alpha=.2)


ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')
title(dataname)
xlim(1e-5,xmax=1e-2)

legend();

In [ ]:
lowerpeaksparam[0] - np.sqrt(lowerpeakspcov[0,0])

In [ ]:
plt.imshow(abs(lowerpeakspcov), cmap="gray", interpolation="nearest", vmin=0)
plt.colorbar()
plt.title('Covariance matrix, lower peaks fit')
plt.show()
print("It's ok to use the uncertainties as long as there aren't strong off-diagonal values.")

plt.imshow(abs(upperpeakspcov), cmap="gray", interpolation="nearest", vmin=0)
plt.colorbar()
plt.title('Covariance matrix, upper peaks fit')
plt.show()
print("It's ok to use the uncertainties as long as there aren't strong off-diagonal values.")

In [ ]:
NDlist

In [ ]:
peaksPL

In [ ]:
os.chdir(analysisfolder)
today = datetime.date.isoformat(datetime.date.today())
if not os.path.exists('taken-' + datadate):
    os.makedirs('taken-' + datadate)
os.chdir('taken-' + datadate)

print('today: '+today)
print os.getcwd()

#cmap = sns.cubehelix_palette(n_colors = 1, start=0, rot=1, light=1, as_cmap=True)
cmap = sns.cubehelix_palette( light=1, as_cmap=True)
#cmap = sns.color_palette("Grey", as_cmap=True)

figure(figsize=(8,3))
plt.pcolormesh(powlistwrap[::-1],  binsequence, transpose(powertabledata), cmap=cmap)
sns.despine()
cbar = colorbar(drawedges=False)
cbar.set_label('Probability')



#ylim(ymin=0)
ax = plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')

xticks(powlist, [power*1000*1000 for power in powlist],rotation='vertical')
#xticks(powlist, [pow for pow in powlist],rotation='vertical')

plt.ylabel(PLname)
plt.xlabel('Laser power into objective ($\mathrm{\mu}$W)')
#plt.xlabel('Laser power into objective (W)')
title(dataname)


# Cover section that has no data
plt.axvline(x=1.3e-4, linewidth=80, color='ghostwhite')

# plot curvefits
plot(x,lowerpeaksfit, '-', color='lightgrey')
plot(x,upperpeaksfit, '-', color='lightgrey') 

# plot points at the peaks
for i in range(len(NDlist)):
    plot([powlist[i]]*len(peaksPL.get(NDlist[i])),peaksPL.get(NDlist[i]), '.', color='grey')

print('Lower peaks saturation at ' + str(lowerpeaksparam[0]*1000*1000)
      + ' ± ' + str(np.sqrt(lowerpeakspcov[0,0])) + ' μW')
print('Upper peaks saturation at ' + str(upperpeaksparam[0]*1000*1000)
      + ' ± ' + str(np.sqrt(upperpeakspcov[0,0])) + ' μW')

#savefig
plt.savefig(today + ',' + dataname+',doublesaturationcurve.pdf', bbox_inches='tight')
plt.savefig(today + ',' + dataname+',doublesaturationcurve.png', bbox_inches='tight', dpi=200)
print(today + ',' + dataname+',doublesaturationcurve')

In [ ]:
## The heatmap isn't as customizable but it's more automated 
## so it's a good check that I entered in the variables correctly.
sns.heatmap(transpose(powertabledata), cmap = cmap)
ylabel(PLname)
title(dataname)

## decrease number of ticks on x axis.
ax = plt.gca()
#ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(100))
#ax.yaxis.set_major_formatter(mpl.ticker.ScalarFormatter())

In [ ]:
## This cell doesn't have to be run. Just some thoughts about the data system.

figure(figsize(5,3))
plt.pcolormesh(data)
sns.despine()

cbar =colorbar()
cbar.set_label(PLname)
title('pcolormesh example')
ylabel('Time (a.u.)')
xlabel('dataset')
plt.show()

figure()
sns.heatmap(data, linewidths=0)
title('heatmap example')
ylabel('Time (a.u.)')
xlabel('ND')


# https://stackoverflow.com/questions/49020709/seaborn-heatmap-with-numerical-axes
# You cannot "convince" heatmap not to produce a categorical plot. 
# Best use another image plot, which uses numerical axes. For example, use a pcolormesh plot. 
# The assumption is of course that the columns and rows are equally spread. 


## Some thoughts about figure spacing
plt.figure(figsize(6,1))
for ND in NDlist:
    plt.axvline(x=ND)
sns.despine()
xlabel('ND')
plt.gca().invert_xaxis()
    
plt.figure()
for thispower in powlist:
   # print thispower
    plt.axvline(x=thispower)
#plt.xlim(xmin=min(powlist), xmax = max(powlist))
sns.despine()
ax = plt.gca()
ax.set_xscale('log')
xlabel('measured power (W)' )    
    
plt.figure()
for ND in NDlist:
    plt.axvline(x=10**-ND)
plt.xlim(xmax=10**-min(NDlist))   
sns.despine()
xlabel('10^-ND')

plt.figure()
for thispower in powlist:
  #  print thispower
    plt.axvline(x=thispower)
plt.xlim(xmin=min(powlist), xmax = max(powlist))
sns.despine()
xlabel('measured power (W)' )




plt.show()